In [71]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import datetime
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [72]:
from ce_generator import CE5min, CE15min

In [74]:
# Check dataset sanity

n_train_data = 10000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_train_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [75]:
# Generate and save training data

# _, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
train_data = ae_labels
train_labels = ce_labels
# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

In [84]:
idx=8231
train_data[idx], train_labels[idx]

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

{0: 4999, 1: 1667, 2: 1667, 3: 1667}

In [85]:
np.save('./CE_dataset/nar_5min_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/nar_5min_train_labels_{}.npy'.format(n_train_data), train_labels)

In [89]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [90]:
# Generate and save test data

_, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
test_data = ae_labels
test_labels = ce_labels

In [91]:
np.save('./CE_dataset/nar_5min_test_data.npy', test_data)
np.save('./CE_dataset/nar_5min_test_labels.npy', test_labels)

In [17]:
idx=830
test_data[idx], test_labels[idx]

(array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [18]:
actions,action_labels

(['walk',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'flush_toilet',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk'],
 [7,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  4,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7])

In [38]:
sensor_test_data = np.load('./CE_dataset/ce5min_test_data.npy')
sensor_test_labels = np.load('./CE_dataset/ce5min_test_labels.npy')

In [41]:
sensor_train_data =np.load('./CE_dataset/ce5min_train_data_2000.npy')

In [42]:
sensor_train_data

(2000, 60, 128)

In [39]:
idx=0
sensor_test_data[idx], sensor_test_data[idx]

(array([[0.02952732, 0.7663239 , 0.        , ..., 0.04166073, 0.07221995,
         0.15755884],
        [0.        , 0.7742814 , 0.        , ..., 0.12734969, 0.0987526 ,
         0.2630577 ],
        [0.        , 0.55041856, 0.00659789, ..., 0.21991117, 0.127577  ,
         0.4599582 ],
        ...,
        [0.        , 0.76084113, 0.        , ..., 0.11863101, 0.10553318,
         0.25677156],
        [0.        , 0.        , 0.05281916, ..., 0.02155555, 0.27263078,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.03247079, 0.23547615,
         0.10722303]], dtype=float32),
 array([[0.02952732, 0.7663239 , 0.        , ..., 0.04166073, 0.07221995,
         0.15755884],
        [0.        , 0.7742814 , 0.        , ..., 0.12734969, 0.0987526 ,
         0.2630577 ],
        [0.        , 0.55041856, 0.00659789, ..., 0.21991117, 0.127577  ,
         0.4599582 ],
        ...,
        [0.        , 0.76084113, 0.        , ..., 0.11863101, 0.10553318,
         0.25677156

In [96]:
# Check dataset sanity

n_train_data = 1000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [97]:
# Generate and save training data

ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

id = np.random.randint(n_train_data)
print(ce_data[id], ce_labels[id], ae_labels[id])


# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

[[0.         0.59752786 0.02698135 ... 0.13239703 0.21743889 0.27447048]
 [0.         0.12701517 0.         ... 0.02399664 0.485571   0.        ]
 [0.         0.55041856 0.00659789 ... 0.21991117 0.127577   0.4599582 ]
 ...
 [0.         1.7309957  0.         ... 0.23885222 0.61413485 0.3515493 ]
 [0.         1.9383442  0.         ... 0.25252607 0.5878786  0.41135514]
 [0.         0.         0.         ... 0.23146436 0.11484321 0.10708833]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 3 5 3 5 5 3 5 5 3 5 5 3 5 3 3
 5 3 3 5 3 3 5 3 3 5 5 3 3 5 3 3 5 3 5 3 5 5 3]


{0: 498, 1: 167, 2: 167, 3: 168}

In [98]:
train_data = ce_data
train_labels = ce_labels
np.save('./CE_dataset/ce5min_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/ce5min_train_labels_{}.npy'.format(n_train_data), train_labels)

In [41]:
a = torch.rand(10,4)
a = torch.max(a, 1).values
b = []
b.append(a)
b.append(a)
b=np.concatenate(b)
b.shape

(20,)

In [57]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [59]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)
test_data = ce_data
test_labels = ce_labels
test_ae_labels = ae_labels

id = np.random.randint(n_test_data)
print(ce_data[id], ce_labels[id], ae_labels[id])


# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

[[0.         0.24287072 0.         ... 0.21562669 0.         0.25501838]
 [0.         0.5500595  0.         ... 0.08163978 0.01720797 0.21167888]
 [0.         2.849454   0.         ... 0.9384143  0.6820142  1.0807836 ]
 ...
 [0.         0.         0.         ... 0.24441478 0.         0.        ]
 [0.         0.5032877  0.         ... 0.49155915 0.31655455 0.26569912]
 [0.         0.7639491  0.         ... 0.41628844 0.39335656 0.1756933 ]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 8 3 3 5 3 5 5 3 3 5 3 3 5 3 3 5 5 3 5 5 3 3 5 5 3 5 3 3 5 5 3 5 3 3 5
 5 3 5 3 5 5 3 3 5 3 3 5 5 3 5 3 3 5 5 3 3 5 5]


{0: 499, 1: 167, 2: 167, 3: 167}

In [44]:
np.save('./CE_dataset/ce5min_test_data.npy', test_data)
np.save('./CE_dataset/ce5min_test_labels.npy', test_labels)
np.save('./CE_dataset/ce5min_test_ae_labels.npy', test_ae_labels)

In [56]:
id = 0
test_ae_labels[id], test_labels[id]

(array([7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        4, 4, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
([7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [92]:
# Check dataset sanity

n_val_data = 2000
ce5 = CE5min(n_val_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

# ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=True)
# print(ce_data.shape, ce_labels.shape, ae_labels.shape)
# print(ce_labels[np.random.randint(n_val_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'

In [93]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

In [94]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.         0.         0.         ... 0.06528988 0.2113285  0.12570032]
 [0.         1.0605559  0.         ... 0.34247494 0.1836755  0.4782271 ]
 [0.         0.         0.         ... 0.03245701 0.23596603 0.10697177]
 ...
 [0.         0.3825536  0.         ... 0.15879856 0.19710334 0.199824  ]
 [0.         0.         0.         ... 0.39903414 0.         0.1900885 ]
 [0.         0.44626647 0.         ... 0.06482062 0.17168573 0.22844875]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 7 8 8 3 5 3 5 5 3 3 5 3 5 3 5 3 5 3 3 5 3 3 5 5 3 3 5 5 3 3 5 5 3 3 5
 3 3 5 3 3 5 5 3 3 5 3 3 5 3 5 3 3 5 3 3 5 3 5]


(2000, 60, 128)

In [95]:
np.save('./CE_dataset/nar_5min_val_data.npy'.format(n_val_data), ae_labels)
np.save('./CE_dataset/nar_5min_val_labels.npy'.format(n_val_data), ce_labels)

In [64]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'

In [65]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

In [66]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.         0.52397037 0.         ... 0.26281708 0.08927657 0.48899636]
 [0.         0.8499469  0.         ... 0.12900618 0.03549089 0.27828065]
 [0.         0.67935085 0.         ... 0.13285539 0.265935   0.11566477]
 ...
 [0.         0.         0.014309   ... 0.06861457 0.3505727  0.16545737]
 [0.0288349  0.7667486  0.         ... 0.04468061 0.06900876 0.16003235]
 [0.         0.15812388 0.         ... 0.17547974 0.15356036 0.3974081 ]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 7 7 7 7 7 7 7 7 5 5 5 5 5 5 4 4 4 8 8 8 8 8 8 8 8 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


(1000, 60, 128)

In [67]:
np.save('./CE_dataset/ce5min_val_data.npy'.format(n_val_data), ce_data)
np.save('./CE_dataset/ce5min_val_labels.npy'.format(n_val_data), ce_labels)

In [8]:
n_train_data = 20000
ce5 = CE5min(n_train_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [9]:
state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)


In [10]:
print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_train_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

(20000, 60, 128) (20000, 60) (20000, 60) (20000, 60) (20000, 60)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[2 2 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [11]:
np.savez_compressed('./CE_dataset/state_nar_5min_full_train_{}.npz'.format(n_train_data), data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

In [13]:
n_test_data = 1000
ce5 = CE5min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_nar_5min_full_test.npz', data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'

In [25]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_val_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_nar_5min_full_val.npz', data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'

In [20]:
n_train_data = 4000
ce5 = CE5min(n_train_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_train_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_train_{}.npz'.format(n_train_data), data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [24]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_val_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_val.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'

In [22]:
n_test_data = 1000
ce5 = CE5min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'

In [50]:
n_test_data = 1000
ce15 = CE15min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce15.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce15.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']

In [51]:
count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

{0: 659, 1: 102, 2: 122, 3: 117}

In [52]:
np.savez_compressed('./CE_dataset/state_ce15min_full_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)
# np.savez_compressed('./CE_dataset/state_ce15min_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

In [3]:
n_test_data = 1000
ce15 = CE15min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce15.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']

In [4]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce15.generate_CE_dataset(is_positive_sample=False)

id = np.random.randint(n_test_data)
print(ce_data[id], ce_labels[id], ae_labels[id])

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

[[0.         0.16167092 0.         ... 0.0583975  0.23000851 0.22904184]
 [0.         0.42456886 0.         ... 0.01147    0.33205035 0.02637389]
 [0.         0.         0.         ... 0.23939635 0.06111969 0.35643005]
 ...
 [0.         0.53951406 0.         ... 0.50156546 0.33773607 0.25719717]
 [0.         0.         0.         ... 0.30418372 0.14125745 0.        ]
 [0.         0.46658438 0.         ... 0.573947   0.3086093  0.42326036]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 3 3 5 5 3 3 5 5 3 5 5 3 3 5 3 5 3 5 3 3 5 3
 5 5 3 5 3 5 5 3 5 5 3 3 5 5 3 3 5 5 3 5 3 5 3 5 5 3 5 5 3 5 5 3 3 5 3 3 5
 5 3 3 5 3 3 5 5 3 5 3 5 5 3 5 5 3 5 3 3

{0: 499, 1: 167, 2: 167, 3: 167}

In [31]:
np.save('./CE_dataset/ce15min_full_test_data.npy', ce_data)
np.save('./CE_dataset/ce15min_full_test_labels.npy', ce_labels)